In [1]:
import pandas as pd
import yfinance as yf
import sqlalchemy
import yfinance as yf
import datetime as dt
import ccxt
import os
import time
import pandas_ta as ta
from backtesting import Backtest, Strategy
import numpy as np
from backtesting.lib import resample_apply
from pandas_datareader import data

C:\Users\ahsua\anaconda3\lib\site-packages\backtesting\_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
exchange=ccxt.currencycom()
markets= exchange.load_markets()

In [3]:
symbols1=pd.read_csv('csymbols.csv',header=None)
symbols=symbols1.iloc[:,0].to_list()
symbols

['AAL/USD',
 'IBM/USD',
 'AMD/USD',
 'MOS/USD',
 'TWTR/USD',
 'CSCO/USD',
 'MSFT/USD',
 'QCOM/USD',
 'GOOGL/USD',
 'TSLA/USD',
 'HPQ/USD',
 'VZ/USD',
 'BA/USD',
 'META/USD',
 'GM/USD',
 'PFE/USD',
 'DAL/USD',
 'DIS/USD',
 'KO/USD',
 'XOM/USD',
 'MU/USD',
 'INTC/USD',
 'SBUX/USD',
 'EBAY/USD',
 'NVDA/USD',
 'T/USD',
 'V/USD',
 'F/USD',
 'MRNA/USD',
 'CRM/USD',
 'LUMN/USD',
 'OXY/USD',
 'CCL/USD',
 'WMT/USD',
 'AAPL/USD',
 'PYPL/USD',
 'AMZN/USD',
 'JPM/USD',
 'BAC/USD',
 'NEM/USD',
 'JNJ/USD',
 'ATVI/USD',
 'ATA/EUR',
 'ADS.DE/EUR',
 'VOW3/EUR',
 'FR40/EUR',
 'UN01/EUR',
 'TUI1/EUR',
 'DBK/EUR',
 'AFP/EUR',
 'LHA/EUR',
 'EU50/EUR',
 'O2D/EUR',
 'DE40/EUR',
 'EUR/USD',
 'AUD/USD',
 'USD/CAD',
 'GBP/USD',
 'USD/CHF',
 'USD/JPY',
 'NZD/USD',
 'ETH/USD',
 'BTC/USD',
 'DXY/USD',
 'EU50/EUR',
 'CN50/USD',
 'US100/USD',
 'US500/USD',
 'CORN/USD',
 'SOYBEAN/USD',
 'US COTTON/USD',
 'COFFEE ARABICA/USD',
 'COPPER/USD',
 'PLATINUM/USD',
 'OIL - CRUDE/USD',
 'OIL - BRENT/USD',
 'NATURAL GAS/USD',


In [4]:
data = []
index = 1
start = time.perf_counter()
for ticker in symbols:
    print(index,ticker,end="\r")
    index += 1
    try:
        data2= exchange.fetch_ohlcv(ticker, timeframe='1d',limit=251)
        header = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
        dfc = pd.DataFrame(data2, columns=header)
        dfc['Date'] = pd.to_datetime(dfc['Date'],unit='ms')
        data.append(dfc)
    except Exception as e:
        print(ticker,e)
end = time.perf_counter()
print(end - start) 

TWTR/USD currencycom does not have market symbol TWTR/USD
38.96696540000001DUSD


In [5]:
bdata= []
index2 = 1
bsymbols1=pd.read_csv('bsymbols.csv',header=None)
bsymbols=bsymbols1.iloc[:,0].to_list()
for bticker in bsymbols:
    print(index2,bticker,end="\r")
    index2 += 1
    df=yf.download(bticker,period="1y")
    df2=df.drop('Adj Close', 1)
    df3=df2.reset_index()
    df4=df3.round(2)
    bdata.append(df4)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [6]:
fullname=symbols1.iloc[:,1].to_list()

In [7]:
engine=sqlalchemy.create_engine('sqlite:///günlük.db')

In [8]:
start = time.perf_counter()
for frame,symbol in zip(data,fullname):
    frame.to_sql(symbol,engine, if_exists='replace')
    #print(symbol)
    #print(frame)
end = time.perf_counter()
print(end - start) 

4.867674700000009


In [9]:
for bframe,bsymbol in zip(bdata,bsymbols):
    bframe.to_sql(bsymbol,engine, if_exists='replace') 
    #print(bframe)

In [10]:
engine=sqlalchemy.create_engine('sqlite:///günlük.db')

In [11]:
def getdailydata(symbol):
    frame=pd.DataFrame(exchange.fetch_ohlcv(symbol, timeframe='1d',limit=10))
    frame = frame[[0,1,2,3,4,5]]
    frame.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
    frame.Date=pd.to_datetime(frame.Date,unit='ms')
    frame.set_index('Date',inplace=True)
    frame=frame.astype(float)
    return frame
getdailydata('BTC/USD')

,Open,High,Low,Close,Volume
Date,,,,,
2022-11-18,16682.10,16986.10,16532.45,16681.60,203318.0
2022-11-19,16681.70,16800.20,16535.20,16686.05,132756.0
2022-11-20,16686.15,16736.80,16196.70,16253.45,224775.0
2022-11-21,16253.60,16280.70,15457.95,15756.25,322006.0
2022-11-22,15756.65,16294.95,15597.70,16205.00,459526.0
2022-11-23,16204.05,16685.35,16145.80,16587.20,418917.0
2022-11-24,16586.60,16794.35,16451.85,16586.45,217451.0
2022-11-25,16586.45,16607.45,16331.95,16509.45,192249.0
2022-11-26,16509.20,16687.70,16375.20,16451.10,146968.0


In [12]:
index = 1
for symbol in symbols1.iloc[:,0]:
    print(index,symbol,end="\r")
    index += 1
    try:
        df=getdailydata(symbol)
        df.to_sql(symbol,engine)
    except Exception as e:
        print(symbol,e)
    

AAL/USD Table 'AAL/USD' already exists.
IBM/USD Table 'IBM/USD' already exists.
AMD/USD Table 'AMD/USD' already exists.
MOS/USD Table 'MOS/USD' already exists.
TWTR/USD currencycom does not have market symbol TWTR/USD
CSCO/USD Table 'CSCO/USD' already exists.
QCOM/USD Table 'QCOM/USD' already exists.
GOOGL/USD Table 'GOOGL/USD' already exists.
TSLA/USD Table 'TSLA/USD' already exists.
HPQ/USD Table 'HPQ/USD' already exists.
VZ/USD Table 'VZ/USD' already exists.
GM/USD Table 'GM/USD' already exists.
PFE/USD Table 'PFE/USD' already exists.
DAL/USD Table 'DAL/USD' already exists.
DIS/USD Table 'DIS/USD' already exists.
KO/USD Table 'KO/USD' already exists.
XOM/USD Table 'XOM/USD' already exists.
MU/USD Table 'MU/USD' already exists.
INTC/USD Table 'INTC/USD' already exists.
SBUX/USD Table 'SBUX/USD' already exists.
EBAY/USD Table 'EBAY/USD' already exists.
T/USD Table 'T/USD' already exists.
F/USD Table 'F/USD' already exists.
LUMN/USD Table 'LUMN/USD' already exists.
OXY/USD Table 'OXY/U

In [1]:
from backtesting import Backtest
from backtesting.lib import crossover, TrailingStrategy

def sup1(fr):
    return ta.supertrend(fr.High, fr.Low, fr.Close, length=10, multiplier=1.0)
def sup2(fr):
    return ta.supertrend(fr.High, fr.Low, fr.Close, length=10, multiplier=2.0)
def sup3(fr):
    return ta.supertrend(fr.High, fr.Low, fr.Close, length=10, multiplier=3.0)


class RsiOscillator(TrailingStrategy):

    # Do as much initial computation as possible
    def init(self):
        
        super().init()
        super().set_trailing_sl(1)
        
        self.atr= self.I(ta.atr, high= self.data.High.s,low= self.data.Low.s, 
                            close= self.data.Close.s, length=10)
        self.super1 = self.I(ta.supertrend, high= self.data.High.s,low= self.data.Low.s, 
                             close= self.data.Close.s, length=10, multiplier=1.0, plot=True, overlay=True)
        self.super2 = self.I(ta.supertrend, high= self.data.High.s,low= self.data.Low.s, 
                             close= self.data.Close.s, length=10, multiplier=2.0,  plot=True, overlay=True)
        self.super3 = self.I(ta.supertrend, high= self.data.High.s,low= self.data.Low.s, 
                             close= self.data.Close.s, length=10, multiplier=3.0,  plot=True, overlay=True)
        self.super1_w = resample_apply(
            'W-FRI',sup1, self.data.df,plot=False)
        self.super2_w = resample_apply(
            'W-FRI',sup2, self.data.df, plot=False)
        self.super3_w = resample_apply(
            'W-FRI',sup3, self.data.df,plot=False)        
        self.ema20 = self.I(ta.ema, close= self.data.Close.s, length=20, plot=False)
        self.ema20_w = resample_apply('W-FRI',ta.ema, self.data.Close.s, length=20, plot=False)
        self.ema50 = self.I(ta.ema, close= self.data.Close.s, length=50)
        self.ema200 = self.I(ta.ema, close= self.data.Close.s, length=200)
        self.ema50_w = resample_apply('W-FRI',ta.ema, self.data.Close.s, length=50, plot=False)
        self.macd = self.I(ta.macd, close= self.data.Close.s, plot=False)
        self.macd_w = resample_apply('W-FRI',ta.macd, self.data.Close.s, plot=False)
    # Step through bars one by one
    # Note that multiple buys are a thing here
    def next(self):
        
        super().next()
        price = self.data.Close.s[-1]
        atr = self.atr[-1]
        super1=self.super1[0]
        super2=self.super2[0]
        super3=self.super3[0]
        super1w=self.super1_w[0]
        super2w=self.super2_w[0]
        super3w=self.super3_w[0]
        macdh=self.macd[1]
        macdh_w=self.macd_w[1]
        if (price>self.ema50_w[-1]):
            if (super2[-2]==super2[-1]) and (super3[-2]==super3[-1])\
            and price>super1[-1]:
                #self.position.close()
                self.buy(size=1, sl=price-atr)
        #if price<self.super1[-1]:
        #        self.position.close()
        #        #self.sell()

bt = Backtest(df, RsiOscillator, cash=700, commission=.002)

stats = bt.run()

bt.plot(plot_volume=False)
stats

C:\Users\ahsua\anaconda3\lib\site-packages\backtesting\_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

NameError: name 'df' is not defined

In [ ]:
def BT(interval):
    return=[]
    for symbol in symbols1.iloc[:,0]:
        qry=f"""SELECT*FROM '{symbol}'
        WHERE 

